In [1]:
from tensorflow.python.keras.engine import data_adapter

def _is_distributed_dataset(ds):
    return isinstance(ds, data_adapter.input_lib.DistributedDatasetSpec)

data_adapter._is_distributed_dataset = _is_distributed_dataset

In [2]:
import os
import pandas as pd
import numpy as np

import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Input

def build_autoencoder(input_dim):
    input_layer = Input(shape=(input_dim,))
    encoded = Dense(64, activation='relu')(input_layer)
    encoded = Dense(32, activation='relu')(encoded)
    decoded = Dense(64, activation='relu')(encoded)
    decoded = Dense(input_dim, activation='sigmoid')(decoded)
    
    autoencoder = Model(inputs=input_layer, outputs=decoded)
    encoder = Model(inputs=input_layer, outputs=encoded)
    
    autoencoder.compile(optimizer='adam', loss='mse')
    return autoencoder, encoder

def load_and_preprocess_data(folder_path):
    all_data = []
    file_list = os.listdir(folder_path)
    for file_name in file_list:
        if file_name.endswith('.csv'):
            file_path = os.path.join(folder_path, file_name)
            df = pd.read_csv(file_path)
            all_data.append(df.values)
    all_data = np.concatenate(all_data, axis=0)
    return all_data

# Feature extraction
def feature_extraction(folder_path):
    data = load_and_preprocess_data(folder_path)
    input_dim = data.shape[1]
    autoencoder, encoder = build_autoencoder(input_dim)
    autoencoder.fit(data, data, epochs=1, batch_size=512, shuffle=True)
    features = encoder.predict(data)
    return features

folder_path = r'C:\Users\bahad\PycharmProjects\kpmakale\dataset_final2'  
features = feature_extraction(folder_path)
print("Extracted features shape:", features.shape)


75979/75979 ━━━━━━━━━━━━━━━━━━━━ 95s 1ms/step - loss: 7.8451e-05
  37272/1215652 ━━━━━━━━━━━━━━━━━━━━ 13:10 671us/step

KeyboardInterrupt: 

In [ ]:
def load_and_preprocess_data_with_labels(folder_path):
    all_data = []
    labels = []
    file_list = os.listdir(folder_path)
    
    for file_name in file_list:
        if file_name.endswith('.csv'):
            file_path = os.path.join(folder_path, file_name)
            df = pd.read_csv(file_path)
            all_data.append(df.values)
            
            # Create label based on filename
            if "_H_" in file_name:
                labels.append(1)  # Hypertension
            elif "_N_" in file_name:
                labels.append(0)  # Healthy
                
    all_data = np.concatenate(all_data, axis=0)
    labels = np.array(labels).repeat(df.shape[0])  # Repeat label for each row in the file
    
    return all_data, labels


In [ ]:
# Feature extraction with labels
def feature_extraction_with_labels(folder_path):
    data, labels = load_and_preprocess_data_with_labels(folder_path)
    input_dim = data.shape[1]
    autoencoder, encoder = build_autoencoder(input_dim)
    autoencoder.fit(data, data, epochs=5, batch_size=512, shuffle=True)
    features = encoder.predict(data)
    return features, labels


In [ ]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import LSTM
folder_path = r'C:\Users\bahad\PycharmProjects\kpmakale\dataset_final2'
features, labels = feature_extraction_with_labels(folder_path)
print("Extracted features shape:", features.shape)
print("Labels shape:", labels.shape)

# Reshape features for LSTM input
num_samples = features.shape[0]
num_features = features.shape[1]
lstm_input = features.reshape((num_samples, 1, num_features))

# Split the data
X_train, X_test, y_train, y_test = train_test_split(lstm_input, labels, test_size=0.2, random_state=42)

# Build LSTM model
model = Sequential()
model.add(LSTM(64, input_shape=(1, num_features), activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='sigmoid')) 

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

# Train the model
model.fit(X_train, y_train, epochs=5, batch_size=128, validation_data=(X_test, y_test))

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)

# Make predictions
predictions = model.predict(X_test)
